# Working with the Archive Query Log in Elasticsearch
## Hands-On Session

This notebook guides you through working with the Archive Query Log (AQL) data in Elasticsearch to support your lab projects. Throughout, links to the [🔗 official ElasticSearch documentation](https://www.elastic.co/docs/reference/) are provided to help you start at the right place with your own coding.


## 1. Setting Up the Environment

Elasticsearch is a distributed search and analytics engine that allows us to efficiently query and analyze the Archive Query Log (AQL) data. The ElasticSearch cluster used in this course offers 2 separate search indices: one for SERPs (i.e. queries), and one for results (i.e., the snippets extracted from SERPs).

First, let's import the necessary libraries and connect to Elasticsearch.

In [1]:
import json
import pandas as pd
import seaborn as sns
from elasticsearch import Elasticsearch

Second, lets supply all the information necessary and connect to the Elasticsearch cluster.

**⚠️IMPORTANT**: you need to be connected to the Webis VPN to access ElasticSearch! Follow [🔗 the instructions](https://temir.org/teaching/scalable-language-technologies-ss25/vpn-instructions.html) to establish a connection after you obtained the VPN certificate from the teaching staff.

In [2]:
# Connection parameters
ES_HOST = "https://elasticsearch.bw.webis.de:9200"
ES_API_KEY = "<KEY>" # Replace with your own key
INDEX_NAME_SERPS = "aql_serps"
INDEX_NAME_RESULTS = "aql_results"

In [3]:
es = Elasticsearch(ES_HOST, api_key=ES_API_KEY, verify_certs=True, request_timeout=30)
# Test connection
try:
    es.search(index=INDEX_NAME_SERPS, body={"query": {"match_all": {}}})
    print("✅Successfully connected to Elasticsearch!")
except:
    print("❌ Connection to Elasticsearch failed.")

❌ Connection to Elasticsearch failed.


## 2. Index Mappings

A good first point to start is to explore the index mappings – they define the names and datatypes of all fields available in the index.

*Note: the info below is pasted as string, instead of retrieved from the API, as the keys distributed in the lab do not have permission to view index mappings. Execute the cell to pretty-print it.*

In [4]:
print(json.dumps(json.loads('{"aql_serps": {"mappings": {"properties": {"archive": {"properties": {"cdx_api_url": {"type": "keyword"}, "id": {"type": "keyword"}, "memento_api_url": {"type": "keyword"}, "priority": {"type": "rank_feature"}}}, "capture": {"properties": {"digest": {"type": "keyword"}, "id": {"type": "keyword"}, "mimetype": {"type": "keyword"}, "status_code": {"type": "integer"}, "timestamp": {"type": "date", "format": "strict_date_time_no_millis"}, "url": {"type": "keyword"}}}, "last_modified": {"type": "date", "format": "strict_date_time_no_millis"}, "provider": {"properties": {"domain": {"type": "keyword"}, "id": {"type": "keyword"}, "priority": {"type": "rank_feature"}, "url_path_prefix": {"type": "keyword"}}}, "url_offset": {"type": "integer"}, "url_offset_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "url_page": {"type": "integer"}, "url_page_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "url_query": {"type": "text"}, "url_query_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "warc_direct_answers": {"type": "nested", "properties": {"id": {"type": "keyword"}}}, "warc_direct_answers_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "warc_downloader": {"properties": {"id": {"type": "keyword"}, "last_downloaded": {"type": "date", "format": "strict_date_time_no_millis"}, "should_download": {"type": "boolean"}}}, "warc_location": {"properties": {"file": {"type": "keyword"}, "length": {"type": "long"}, "offset": {"type": "long"}}}, "warc_query": {"type": "text"}, "warc_query_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "warc_snippets": {"type": "nested", "properties": {"id": {"type": "keyword"}, "rank": {"type": "integer"}}}, "warc_snippets_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}}}}}'), indent=4))

{
    "aql_serps": {
        "mappings": {
            "properties": {
                "archive": {
                    "properties": {
                        "cdx_api_url": {
                            "type": "keyword"
                        },
                        "id": {
                            "type": "keyword"
                        },
                        "memento_api_url": {
                            "type": "keyword"
                        },
                        "priority": {
                            "type": "rank_feature"
                        }
                    }
                },
                "capture": {
                    "properties": {
                        "digest": {
                            "type": "keyword"
                        },
                        "id": {
                            "type": "keyword"
                        },
                        "mimetype": {
                            "type": "keyword"
        

For the SERP index, five keys (and their subfields) are of interest to us:
- `capture.*` - including information about how the URL was originally recorded in the Wayback machine, e.g., timestamp
- `provider.*` - including information about the search provider this SERP was crawled from, e.g., domain
- `url_query` - the query as parsed from the URL string
- `warc_query` - the query as parsed from the web page source (WARC is the an archive format websites are available in)
- `warc_snippets` - if available, lists the ID and rank of result snippets (not the text though!)

In [5]:
print(json.dumps(json.loads('{"aql_results": {"mappings": {"properties": {"archive": {"properties": {"cdx_api_url": {"type": "keyword"}, "id": {"type": "keyword"}, "memento_api_url": {"type": "keyword"}, "priority": {"type": "rank_feature"}}}, "capture": {"properties": {"digest": {"type": "keyword"}, "id": {"type": "keyword"}, "mimetype": {"type": "keyword"}, "status_code": {"type": "integer"}, "timestamp": {"type": "date", "format": "strict_date_time_no_millis"}, "url": {"type": "keyword"}}}, "last_modified": {"type": "date", "format": "strict_date_time_no_millis"}, "provider": {"properties": {"domain": {"type": "keyword"}, "id": {"type": "keyword"}, "priority": {"type": "rank_feature"}, "url_path_prefix": {"type": "keyword"}}}, "serp": {"properties": {"id": {"type": "keyword"}}}, "snippet": {"properties": {"content": {"type": "text"}, "id": {"type": "keyword"}, "rank": {"type": "integer"}, "text": {"type": "text"}, "title": {"type": "text"}, "url": {"type": "keyword"}}}, "snippet_parser": {"properties": {"id": {"type": "keyword"}, "last_parsed": {"type": "date", "format": "strict_date_time_no_millis"}, "should_parse": {"type": "boolean"}}}, "warc_after_serp_downloader": {"properties": {"id": {"type": "keyword"}, "last_downloaded": {"type": "date", "format": "strict_date_time_no_millis"}, "should_download": {"type": "boolean"}}}, "warc_after_serp_location": {"properties": {"file": {"type": "keyword"}, "length": {"type": "long"}, "offset": {"type": "long"}}}, "warc_before_serp_downloader": {"properties": {"id": {"type": "keyword"}, "last_downloaded": {"type": "date", "format": "strict_date_time_no_millis"}, "should_download": {"type": "boolean"}}}, "warc_before_serp_location": {"properties": {"file": {"type": "keyword"}, "length": {"type": "long"}, "offset": {"type": "long"}}}, "warc_downloader": {"properties": {"id": {"type": "keyword"}, "last_downloaded": {"type": "date", "format": "strict_date_time_no_millis"}, "should_download": {"type": "boolean"}}}, "warc_location": {"properties": {"file": {"type": "keyword"}, "length": {"type": "long"}, "offset": {"type": "long"}}}}}}}'), indent=4))

{
    "aql_results": {
        "mappings": {
            "properties": {
                "archive": {
                    "properties": {
                        "cdx_api_url": {
                            "type": "keyword"
                        },
                        "id": {
                            "type": "keyword"
                        },
                        "memento_api_url": {
                            "type": "keyword"
                        },
                        "priority": {
                            "type": "rank_feature"
                        }
                    }
                },
                "capture": {
                    "properties": {
                        "digest": {
                            "type": "keyword"
                        },
                        "id": {
                            "type": "keyword"
                        },
                        "mimetype": {
                            "type": "keyword"
      

For the result index, four keys are of interest to us:
- `capture.*` - as before, this includes Wayback information
- `provider.*` - as before, this includes search provider information
- `serp.id` - the ID of the SERP this result is part of (can be used to cross-reference with the SERP index)
- `snippet.*` - the snippet information (including ID, title, text, rank)

## 3. Querying

**Term Query**

Let's start with some simple queries to explore the AQL data. We will first do a term query on the SERP index. A *term query* returns documents that contain an exact term in a provided field. You can use the term query to find documents based on a precise value such as a provider domain.

[🔗 Elastic Documentation: Term Query](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-term-query)

In [6]:
# Example 1: Simple term query to find documents for a specific provider
body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
pd.json_normalize(response['hits']['hits'])

AuthenticationException: AuthenticationException(401, 'security_exception', 'unable to authenticate with provided credentials and anonymous access is not allowed for this request')

In [ ]:
# Example 2: Search for queries containing specific terms
body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
pd.json_normalize(response['hits']['hits'])

**Compound Queries**

Multiple queries can be combined to build a unified result set. For example, we can use a boolean compoung query, which wraps a multi-match query to get results on a search topic, and a term query simultaneously to get only results from a certain search provider. Results are combined like with a logical AND – both queries must match in order for a document to appear in the final result.

[🔗 Elastic Documentation: Compound Queries](https://www.elastic.co/docs/reference/query-languages/query-dsl/compound-queries)

[🔗 Elastic Documentation: Boolean Query](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-bool-query)

In [ ]:
# Example 3: Boolean query combining multiple conditions
body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
pd.json_normalize(response['hits']['hits'])

**Other Query Types**

Elastic offers many more specialized query types. You can find them in the [*QueryDSL section*](https://www.elastic.co/docs/reference/query-languages/querydsl) of the documentation. One handy example is the *range query*, which allows us to only get results from a certain value, e.g., time, range. Another type is the *more like this query*, which returns documents that are really similar to the example provided.

[🔗 Elastic Documentation: Range Query](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-range-query)

[🔗 Elastic Documentation: More Like This Query](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-mlt-query)

In [ ]:
# Example 4: Time range query; only results from 2020
body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
pd.json_normalize(response['hits']['hits'])

In [ ]:
# Example 5: Finding similar queries using "more like this"
seed_query = "machine learning"

body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
pd.json_normalize(response['hits']['hits'])

## 4. Aggregation

ElasticSearch not only allows to query data, but also perform aggregations over the result. For example, we can output a histogram of time for the results of a full-text search, for example to investigate the popularity of a query trend over time.

[🔗 Elastic Documentation: Aggregation](https://www.elastic.co/docs/reference/aggregations/)

[🔗 Elastic Documentation: Date Histogram](https://www.elastic.co/docs/reference/aggregations/search-aggregations-bucket-datehistogram-aggregation)

In [ ]:
# Example: Query trends over time
body = {
    # Write your query body. Refer to the docs for hints.
}

response = es.search(index=INDEX_NAME_SERPS, body=body)
response["aggregations"]

In [ ]:
# Visualize the result using seaborn (or your plotting library of choice)

We can also aggregate the distribution of scalar term values like provider domain. Let's compare which providers occur in the two queries `covid` and `lasagna`.

[Elastic Documentation: Terms aggregation](https://www.elastic.co/docs/reference/aggregations/search-aggregations-bucket-terms-aggregation)

In [ ]:
# Example: Domain distribution for query
data = []
for q in ["lasagna", "covid"]:
    pass
    # Write your query body and data processing steps

In [ ]:
# Visualize your results

## 5. Combining Results from Both Indices

Elastic can't perform merges between two indices on its own, but we can create separate queries and combine them locally in pandas. First, we find relevant SERPs, and then, we return all results (if present) from those. While creating these queries keep in mind to:
- query the results index with a SERP id, to avoid having to do separate queries for each individual snippet
- built a filter for the SERP query to restrict it to results which actually have snippets present
- built a filter for the results query to restrict it to results which actually have a text



In [ ]:
# Retrieve relevant SERPs from the SERP index
RAG_QUERY = "population of italy"
body = {
    # Write your query body
}

serps = es.search(index=INDEX_NAME_SERPS, body=body)

In [ ]:
# Retrieve the corresponding result texts from the result index
query = {
    # Write your query body
}
texts = es.search(index=INDEX_NAME_RESULTS, body=query)

With both searches down, we can merge the results in pandas and built a well-structured DataFrame.

In [ ]:
context = None
# Write your data processing to merge the queries from the retrieved SERPs with their snippet texts retrieved from the results index.

## 6. Putting it all together: RAG

As a final exercise, lets try to combine these snippets into a RAG answer. Let python combine all retrieved data into a prompt string that yields good results, and copy it into your favorite LLM. You can also wrap the whole pipeline into function to try out different queries.

In [ ]:
RAG_PROMPT = """
Formulate your own RAG prompt!
"""

print(RAG_PROMPT) # Paste into your LLM of choice and see what happens!